In [1]:
import keras.models
import pandas as pd
import numpy as np
import keras.models
import keras.layers as layers
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential, activations
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import sys

2022-01-09 19:30:59.614950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-09 19:30:59.615032: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_normalization_infos(*x_s, columns):
    normalization_infos = pd.DataFrame(data=[[1000 for _ in range(len(columns))],[0 for _ in range(len(columns))]],
                                       index=["min","max"],
                                       columns=columns)
    for x in x_s :
        for col in columns:
            min_value = min(normalization_infos[col]["min"], x[col].min())
            max_value = max(normalization_infos[col]["max"], x[col].max())
            normalization_infos[col] = [min_value, max_value]

    normalization_infos.loc["spread"] = normalization_infos.apply(lambda c : c["max"] - c["min"], axis=0)

    return normalization_infos


def normalize(x : pd.DataFrame, normalization_infos : pd.DataFrame):
    for col in x.columns:
        x[col] = (x[col] - normalization_infos[col]["min"])/normalization_infos[col]["spread"]
    return x


def m_mape(y_true, y_predict):
    n = len(y_true)
    At = np.array(y_true) + 1
    Ft = np.array(y_predict) + 1

    res = ((100/n)*(np.sum(np.abs((Ft-At)/At))))
    return res


    # return (100 / len(y_true)) * (np.sum(np.abs((np.array(y_predict) + 1 - np.array(y_true) + 1) / np.array(y_true) + 1)))


In [41]:
def create_mlp_regression(x_train, nb_hidden_layers=0, final_activation=None):
    model = keras.models.Sequential()
    model.add(layers.Dense(30, activation="relu", input_shape=x_train.shape[1:]))
    # model.add(layers.Dropout(0.1))
    if nb_hidden_layers > 0:
        for i in range(nb_hidden_layers):
            model.add(layers.Dense(30, activation="relu", input_shape=x_train.shape[1:]))
            # model.add(layers.Dropout(0.1))
    if final_activation is None:
        model.add(layers.Dense(1))
    else :
        exp_relu = lambda x : activations.exponential(activations.relu(x)) - 1
        model.add(layers.Dense(1, activation='relu'))

    # default = learning_rate=0.01, momentum=0.0, nesterov=False
    # momentum: float hyperparameter >= 0 that accelerates gradient descent in the relevant direction and dampens oscillations.
    # nesterov: boolean. Whether to apply Nesterov momentum.

    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0, nesterov=False)
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

# Code for Submission

In [18]:
cols_to_drop = {"day","Id", "station_id","month","longitude_idx","latitude_idx", "next hour precipitation (kg/m^2)", "wind_direction (deg)"}

df_2016 = pd.read_csv("./preprocessed_data_Defi-IA-2022-Rain-Frogs/X_all_2016_final_by_day.zip")
df_2017 = pd.read_csv("./preprocessed_data_Defi-IA-2022-Rain-Frogs/X_all_2017_final_by_day.zip")
df_train = pd.concat([df_2016, df_2017], axis=0)

del df_2017
del df_2016

# df_train["raining"] = df_train["ground_truth"].apply(lambda x : 0 if (x==0) else 1)
x_train, x_valid, y_reg_train, y_reg_valid = train_test_split(df_train.drop(["ground_truth"], axis=1), df_train["ground_truth"], test_size=0.2) # , df_train["raining"]
#drop unnecessary columns
x_train.drop(list(set(x_train.columns)&cols_to_drop), axis=1, inplace=True)
x_valid.drop(list(set(x_train.columns)&cols_to_drop), axis=1, inplace=True)

y_reg_train = np.float32(y_reg_train)
y_reg_valid = np.float32(y_reg_valid)

del df_train

# Evaluation set

x_test = pd.read_csv("./preprocessed_data_Defi-IA-2022-Rain-Frogs/X_all_test_final_by_day.zip")
submission = pd.DataFrame(x_test[["Id"]])

columns = list(x_train.columns)
x_train = x_train[columns]
x_test = x_test[columns]
x_valid = x_valid[columns]

# Normalization
normalization_infos = get_normalization_infos(x_train, x_test, columns=columns)
x_train = normalize(x_train, normalization_infos)
x_test = normalize(x_test, normalization_infos)
x_valid = normalize(x_valid, normalization_infos)


In [42]:
# Regression
model = create_mlp_regression(x_train, nb_hidden_layers=5, final_activation='relu')
model.summary()
history = model.fit(x_train, y_reg_train, epochs=75, validation_data=(x_valid,y_reg_valid), verbose=True)
y_pred = np.concatenate(model.predict(x_test.values), axis=0)
y_pred = y_pred + 1

print(y_pred)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_108 (Dense)           (None, 30)                2940      
                                                                 
 dense_109 (Dense)           (None, 30)                930       
                                                                 
 dense_110 (Dense)           (None, 30)                930       
                                                                 
 dense_111 (Dense)           (None, 30)                930       
                                                                 
 dense_112 (Dense)           (None, 30)                930       
                                                                 
 dense_113 (Dense)           (None, 30)                930       
                                                                 
 dense_114 (Dense)           (None, 1)               

In [44]:
submission = pd.read_csv('./submission_ann_regression_75_square_relu.csv')
print(submission['Prediction'].describe())
submission[["Id","Prediction"]].to_csv("./submission_ann_regression_75_epochs.csv", index=False)

count    85140.000000
mean         1.083525
std          0.081641
min          1.000000
25%          1.024015
50%          1.060697
75%          1.122318
max          1.997445
Name: Prediction, dtype: float64


# Code for experiments

In [135]:
cols_to_drop = {"day","Id", "station_id","month","longitude_idx","latitude_idx", "next hour precipitation (kg/m^2)", "wind_direction (deg)"}

df_train = pd.read_csv("./preprocessed_data_Defi-IA-2022-Rain-Frogs/X_all_2016_final_by_day.zip")
df_train["raining"] = df_train["ground_truth"].apply(lambda x : 0 if (x==0) else 1)
df_train["not_raining"] = 1-df_train["raining"]
x_train, y_reg_train, y_cls_train = df_train.drop(["ground_truth","raining"], axis=1), df_train["ground_truth"], df_train[["not_raining","raining"]].to_numpy()
#drop unnecessary columns
x_train.drop(list(set(x_train.columns)&cols_to_drop), axis=1, inplace=True)
del df_train

df_test = pd.read_csv("./preprocessed_data_Defi-IA-2022-Rain-Frogs/X_all_2017_final_by_day.zip")
df_test["raining"] = df_test["ground_truth"].apply(lambda x : 0 if (x==0) else 1)
df_test["not_raining"] = 1-df_test["raining"]
x_test, y_reg_test, y_cls_test = df_test.drop(["ground_truth","raining"], axis=1), df_test["ground_truth"], df_test[["not_raining","raining"]].to_numpy()
#drop unnecessary columns
x_test.drop(list(set(x_test.columns)&cols_to_drop), axis=1, inplace=True)
del df_test

columns = list(x_train.columns)
x_train = x_train[columns]
x_test = x_test[columns]

normalization_infos = get_normalization_infos(x_train, x_test, columns=columns)

x_train = normalize(x_train, normalization_infos)
x_test = normalize(x_test, normalization_infos)

y_reg_train = np.float32(y_reg_train)
y_reg_test = np.float32(y_reg_test)

## Classification

In [118]:
"""def create_mlp_classification(x_train, nb_hidden_layers=0):
    model = keras.models.Sequential()
    model.add(layers.Dense(30, activation="relu", input_shape=x_train.shape[1:]))
    # model.add(layers.Dropout(0.1))
    if nb_hidden_layers > 0:
        for i in range(nb_hidden_layers):
            model.add(layers.Dense(30, activation="relu", input_shape=x_train.shape[1:]))
            # model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1, activation='softmax'))

    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])
    # model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model"""

# Creating model using the Sequential in tensorflow
def build_model_classf(*layers, loss ='binary_crossentropy', metrics = ("AUC","accuracy"), learning_rate=0.001, last_activation="softmax"):
    model = Sequential()
    model.add(layers[0])
    for layer in layers[1:]:
        model.add(layer)
    model.add(Dense(2, activation=last_activation))

    model.compile(
        loss=loss,
        optimizer=Adam(learning_rate=learning_rate),
        metrics=metrics
    )

    print(model.summary())
    return model

def make_cls_model(*layers,x,y,x_val=None,y_val=None,val_split=0.15, epochs=15, batch_size=64, learning_rate=0.001, last_activation="softmax"):
    # build the model
    model_cls = build_model_classf(*layers, learning_rate=learning_rate, last_activation=last_activation)
    # train the model
    if x_val is None :
        history_cls = model_cls.fit(
            x=x.values,
            y=y,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=val_split
            # validation_data=(x_test,y_cls_test)
        )
    else :
        history_cls = model_cls.fit(
            x=x.values,
            y=y,
            epochs=epochs,
            batch_size=batch_size,
            # validation_split=0.15
            validation_data=(x_val,y_val)
        )
    return history_cls,model_cls

In [122]:
# model_cls = create_mlp_classification(x_train, nb_hidden_layers=5)
history_cls, model_cls = make_cls_model(
    Dense(128, activation="relu",input_dim=x_train.shape[1]),
    Dropout(0.1),
    Dense(64, activation="relu"),
    # Dropout(0.2),
    Dense(32, activation="relu"),
    # Dropout(0.2),
    Dense(32, activation="relu"),
    # Dropout(0.2),
    Dense(16, activation="relu"),
    x=x_train,y=y_cls_train,
    epochs=5,
    batch_size=128
)

model_cls.summary()
history_cls = model_cls.fit(x_train.values, y_cls_train, validation_split=0.15, verbose=True)
y_pred_cls = np.concatenate(model_cls.predict(x_test.values), axis=0 )
print(int(y_pred_cls))

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_279 (Dense)           (None, 128)               12672     
                                                                 
 dropout_29 (Dropout)        (None, 128)               0         
                                                                 
 dense_280 (Dense)           (None, 64)                8256      
                                                                 
 dense_281 (Dense)           (None, 32)                2080      
                                                                 
 dense_282 (Dense)           (None, 32)                1056      
                                                                 
 dense_283 (Dense)           (None, 16)                528       
                                                                 
 dense_284 (Dense)           (None, 2)               

## Regression

In [134]:
model = create_mlp_regression(x_train, nb_hidden_layers=5, final_activation="relu")
model.summary()
history = model.fit(x_train.values, y_reg_train.values, epochs=15, validation_split=0.15, verbose=True)
y_pred = np.concatenate(model.predict(x_test.values), axis=0 )
evaluation_score = m_mape(y_reg_test, y_pred)
print('nb_hidden_layer', 5, 'mape', evaluation_score)

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_299 (Dense)           (None, 80)                7920      
                                                                 
 dense_300 (Dense)           (None, 80)                6480      
                                                                 
 dense_301 (Dense)           (None, 70)                5670      
                                                                 
 dense_302 (Dense)           (None, 60)                4260      
                                                                 
 dense_303 (Dense)           (None, 50)                3050      
                                                                 
 dense_304 (Dense)           (None, 40)                2040      
                                                                 
 dense_305 (Dense)           (None, 1)               

In [145]:
# pip install keras-tuner --upgrade
import keras_tuner as kt

def build_model(hp):
    hp_number_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2]) # , 1e-3, 1e-4])

    model = Sequential()
    for i in range(hp_number_layers):
        model.add(layers.Dense(units=30, activation="relu", input_shape=x_train.shape[1:]))
    model.add(layers.Dense(1, activation='relu'))

    optimizer = Adam(learning_rate=hp_learning_rate)
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

tuner.search(x_train.values, y_reg_train, epochs=5, validation_data=(x_test.values, y_reg_test))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


ValueError: Shapes (30,) and (90,) are incompatible